In [1]:
from tinydb import TinyDB, Query
import os
import json
import numpy as np

class DB:
    def __init__(self, scenario_dir: str, db_name: str) -> None:
        self.scenario_dir = scenario_dir
        self.db_instance = TinyDB(db_name)
        self.tables = {field: self.db_instance.table(field) for field in ["reality", "objects", "rules"]}
        
        # Defaults
        self.OBJECTS = {
        "name": None,
        "id": None,
        "cat": "object",
        "desc": "An object.",
        "initRequire": [],
        "constRequire": [],
        "initGains": [],
        "constGains": [],
        "replace": [],
        "restrict": [],
        "visible": True
    }
    self.RULES = {
        "dismantleGain": 0.5
    }
    self.REALITY = {
        "id": None,
        "type": None,
        "amount": None
    }

class Compiler(DB):
    def __init__(self, scenario_dir: str, db_name: str) -> None:
        super().__init__(scenario_dir, db_name)

    def fill_empty(self, obj: dict, default_vals: dict) -> dict:
        for key in default_vals.keys():
            if key not in obj.keys():
                obj.update({key: default_vals[key]})
        return obj


    def integrate_objects(self, content: list[dict]) -> list[dict]:
        error_handle = []
        for element in content: 
            if len([el for el in element.keys() if el in ["name", "id"]]) == 2:
                content[content.index(element)] = self.fill_empty(element, self.OBJECTS)
            else:
                error_handle.append(json.dumps(element))

        if not error_handle:
            return content
        else:
            err = '\n'.join(error_handle)
            print(f"No mandatory fields id and name in objects: indexes: {err}")
                    

    def compile_scenario(self) -> None:
        valid_scenario_files = [f for f in os.listdir(self.scenario_dir) if f.endswith(".json")]
        
        for file in valid_scenario_files:
            with open(f"{self.scenario_dir}/{file}", "r") as f:
                content = json.loads(f.read())
                for key in content.keys():
                    key = key.lower()
                    
                    if key == "objects":
                        integrated_content = self.integrate_objects(content[key])
                        self.tables[key].insert_multiple(integrated_content)
                    elif key == "rules":
                        self.tables[key].insert(self.fill_empty(content[key], self.RULES))
                    elif key == "reality":
                        for real in content[key]:
                            if not np.isin(list(real.keys()), list(self.REALITY.keys())).all():
                                print("Some reality elements does not fit schema.")
                                exit()
                        self.tables[key].insert_multiple(content[key])


In [2]:
Compiler("scenario", "db.json").compile_scenario()

In [36]:
tt = Query()
x.tables["reality"].insert({"name": "Mikolaj"})

1

In [38]:
x.tables["reality"].search(tt.name == "Mikolaj")

[{'name': 'Mikolaj'}]

In [39]:
x.tables["reality"].remove(tt.name == "Mikolaj")

[1]

In [40]:
x.tables["reality"].search(tt.name == "Mikolaj")

[]

In [42]:
x.tables.keys()

dict_keys(['reality', 'concepts', 'rules'])

In [62]:
[1,2, 3] == [1,2]

False

In [63]:
%pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 28.9 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.
